In [1]:
import requests
import re
import random
from bs4 import BeautifulSoup
import time
import pandas as pd 

## List of random user agents

The scammers website changes depending on if it is accessed from a PC or a mobile. They do this by detecting the User Agent of the incoming request and rending a 404 error if its a non mobile.

So we scrape down a bunch of common User agents from various IOS and Android phones.

In [ ]:
link_to_android_user_agents = 'https://developers.whatismybrowser.com/useragents/explore/operating_system_name/android/'
link_to_ios_user_agents = 'https://developers.whatismybrowser.com/useragents/explore/operating_system_name/ios/'
android_user_agents_table = pd.read_html(link_to_android_user_agents)[0]
ios_user_agents_table = pd.read_html(link_to_ios_user_agents)[0]

android_user_agents_table.head()

,User agent,Software,Software Type,Hardware Type,Popularity
0,Mozilla/5.0 (Linux; U; Android 2.2) AppleWebKi...,Android Browser 4,Web Browser,Mobile,Very common
1,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,Google WebLight Proxy,Application - proxy,Mobile - Phone,Very common
2,Mozilla/5.0 (Linux; Android 6.0.1; RedMi Note ...,Android WebView 68,Browser - In-App Browser,Mobile - Phone,Very common
3,Mozilla/5.0 (Linux; Android 7.1.2; AFTMM Build...,Android WebView 70,Browser - In-App Browser,Mobile,Very common
4,Mozilla/5.0 (Linux; Android 10; Android SDK bu...,Chrome 91,Web Browser,Mobile,Very common
5,Mozilla/5.0 (Linux; Android 6.0; LG-H631 Build...,Chrome 38,Web Browser,Mobile - Phone,Very common
6,Mozilla/5.0 (Linux; Android 7.1.2; AFTMM Build...,Android WebView 59,Browser - In-App Browser,Mobile,Very common
7,Mozilla/5.0 (Linux; Android 9; SM-G960F Build/...,Android WebView 74,Browser - In-App Browser,Mobile - Phone,Very common
8,Mozilla/5.0 (Linux; U; Android 8.1.0; zh-CN; E...,UC Browser 11.9,Web Browser,Mobile,Very common
9,Mozilla/5.0 (Linux; Android 7.0; SM-G892A Buil...,Android WebView 68,Browser - In-App Browser,Mobile - Phone,Very common


In [3]:
android_user_agents = android_user_agents_table.loc[android_user_agents_table['Software Type'] == 'Web Browser']['User agent']
ios_user_agents = ios_user_agents_table.loc[ios_user_agents_table['Software Type'] == 'Web Browser']['User agent']

ios_user_agents = ios_user_agents.reset_index().drop(columns=['index'])
android_user_agents = android_user_agents.reset_index().drop(columns=['index'])

## generate registration number and pac numbers

I dont bank with AIB but I used to, I'm pretty sure that AIB registration number is 8 digits long and the pac is 5.
So I randomly generate a Registration number and PAC code

In [4]:
reg_number = str(random.randint(1000_0000,9999_9999))
pac_numbers = str(random.randint(10_000,99_999))
print(reg_number,'\n',pac_numbers)

11707948 
 51077


## Scrape Irish IP addressses from this website: https://www.nirsoft.net/countryip/ie.html
The scammers also take the IP address of the incoming request, possibly to make it easier for them to block requests from certain organisations eg Gards or IP addresses from outside of Ireland

In [5]:
res = requests.get('https://www.nirsoft.net/countryip/ie.html')
soup = BeautifulSoup(res.content,'lxml')
td_tags = soup.find_all('td') #217.183.255.255

scraped_ip_numbers = []

for td in td_tags:
    ip_address = re.match(r'\d\d\d.\d\d\d.\d\d\d.\d\d\d',td.text)
    if ip_address != None:
        scraped_ip_numbers.append(ip_address.group(0))

In [6]:
bogus_ip_addresses = []

for ip_address in scraped_ip_numbers:
    random_ip_address = ''
    for count, three_digits in enumerate(ip_address.split('.')):
        #print(count)
        randomized_ip_address = int(three_digits) +(count*random.randint(-2,5)) # add or subtract a random number from each ip adress 
        random_ip_address += str(randomized_ip_address)+'.'
    bogus_ip_addresses.append(random_ip_address[:-1])

## Send the crap data to the scammers
The scammers want the IP address, User Agent, PAC and Registration Number, so we randomly generate these data and send them. This should slow the scammers down, as for every geniune person who falls victim to the scam there should be 200 fake PAC codes and Registration codes that I've sent them. 

I also saw that the scammers website sends you to a screen which said that the data you have entered is incorrect. This may be because they want potential victims to put in their creditentials again. Or maybe they have already found a way to verify if its a geniune PAC or Registration number. I dont know, but I guess they just want people to repeat their details. So I added a line of code to repeat sending the creditials about 24% of the time.

In [8]:
def createRandomFormData(i):
    if random.random() < 0.72: # rought 72% of the european market is android phones according to https://www.mobileapps.com/blog/android-vs-ios-market-share#5_Europe
        user_agent = android_user_agents.iloc[100%android_user_agents.size]['User agent']
    else:
        user_agent = ios_user_agents.iloc[100%ios_user_agents.size]['User agent']

    ip_address = bogus_ip_addresses[2*i%len(bogus_ip_addresses)]
    reg_number = str(random.randint(1000_0000,9999_9999))
    pac_numbers = str(random.randint(10_000,99_999))
    form_data = {
        'ip': ip_address ,
        'ua': user_agent,
        'regNumber': reg_number,
        'PACNumber': pac_numbers
        }
    return form_data

{'ip': '188.146.135.252',
 'ua': 'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36',
 'regNumber': '99377604',
 'PACNumber': '65100'}

In [9]:
post_url = 'https://aib.ie1.online/files/action.php?type=login'

for i in range(1,4000):
    form_data = createRandomFormData(random.randint(1,1_000_000))
    post = requests.post(post_url,data = form_data)
    while random.random() < 0.24:
        post = requests.post(post_url,data = form_data) # we redo the post because maybe someone will send their details twice
    print('request number:',i,'form data: ',form_data,'status_code:',post.status_code)

request number: 1 form data:  {'ip': '149.161.253.258', 'ua': 'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36', 'regNumber': '27229492', 'PACNumber': '29068'} status_code: 200
request number: 2 form data:  {'ip': '109.108.137.249', 'ua': 'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36', 'regNumber': '60645017', 'PACNumber': '25924'} status_code: 200
request number: 3 form data:  {'ip': '137.193.265.261', 'ua': 'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36', 'regNumber': '75090433', 'PACNumber': '34268'} status_code: 200
request number: 4 form data:  {'ip': '193.182.111.252', 'ua': 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko)', 'regN